In [1]:
!pip install pydeseq2
from pydeseq2.preprocessing import deseq2_norm
import scipy.stats as stats
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def process_column(df, col_number):
    # Calculate the mean for "G" and "F" rows in the specified column
    g_values = df[df.iloc[:, 0].str.startswith("G")].iloc[:, col_number]
    f_values = df[df.iloc[:, 0].str.startswith("F")].iloc[:, col_number]

    g_mean = g_values.mean()
    f_mean = f_values.mean()

    # Calculate the p-value
    t_stat, p_value = stats.ttest_ind(g_values, f_values)

    # Print the means and p-value
    print(f"Mean for 'Ground Samples' rows: {g_mean}")
    print(f"Mean for 'Flight Samples' rows: {f_mean}")
    print(f"P-value: {p_value}")

    # Replace values based on mean comparison for "G" rows
    df.loc[df.iloc[:, 0].str.startswith("G"), df.columns[col_number]] = g_values.apply(lambda x: 1 if x > g_mean else 0)

    # Replace values based on mean comparison for "F" rows
    df.loc[df.iloc[:, 0].str.startswith("F"), df.columns[col_number]] = f_values.apply(lambda x: 1 if x > f_mean else 0)

    return df

def log_scale_transform(df):
    """Applies log base 2 transformation to all numeric values between sample and env columns for each row."""
    transformed_df = df.copy()
    # Apply log2 transformation to all numeric values between sample and env columns
    transformed_df.iloc[:, 1:-1] = transformed_df.iloc[:, 1:-1].applymap(lambda x: np.log2(x) if x > 0 else np.nan)
    # Update the env column to reflect transformation method
    transformed_df['env'] = transformed_df['env'].apply(lambda x: f"log2")
    transformed_df.iloc[:, 0] = transformed_df.iloc[:, 0].apply(lambda x: f"{x}_log2")
    return transformed_df

def z_score_transform(df):
    """Applies Z-score normalization to all numeric values between sample and env columns for each row."""
    transformed_df = df.copy()
    # Apply Z-score normalization row-wise
    transformed_df.iloc[:, 1:-1] = transformed_df.iloc[:, 1:-1].apply(
        lambda row: (row - row.mean()) / row.std(ddof=0), axis=1
    )
    # Update the env column to reflect transformation method
    transformed_df['env'] = transformed_df['env'].apply(lambda x: f"zscore")
    transformed_df.iloc[:, 0] = transformed_df.iloc[:, 0].apply(lambda x: f"{x}_zscore")
    return transformed_df

def sqrt_transform(df):
    """Applies square root transformation to all numeric values between sample and env columns for each row."""
    transformed_df = df.copy()
    # Apply square root transformation to all numeric values between sample and env columns
    transformed_df.iloc[:, 1:-1] = transformed_df.iloc[:, 1:-1].applymap(lambda x: np.sqrt(x) if x >= 0 else np.nan)
    # Update the env column to reflect transformation method
    transformed_df['env'] = transformed_df['env'].apply(lambda x: f"sqrt")
    transformed_df.iloc[:, 0] = transformed_df.iloc[:, 0].apply(lambda x: f"{x}_sqrt")
    return transformed_df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
unnormalized_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/Unnormalized_data.csv')
unnormalized_data = process_column(unnormalized_data, -3)
unnormalized_data = process_column(unnormalized_data, -2)
unnormalized_data = process_column(unnormalized_data, -1)
unnormalized_data.head()
unnormalized_data.dtypes
for col in unnormalized_data.select_dtypes(include='int64').columns:
    unnormalized_data[col] = unnormalized_data[col].astype('float64')
unnormalized_data.dtypes

Mean for 'Ground Samples' rows: 60.4
Mean for 'Flight Samples' rows: 124.66666666666667
P-value: 0.0023841835789307924
Mean for 'Ground Samples' rows: 56.455035634
Mean for 'Flight Samples' rows: 127.84724976333332
P-value: 0.001059631000889135
Mean for 'Ground Samples' rows: 18.59951911
Mean for 'Flight Samples' rows: 25.710508138333335
P-value: 0.11333998767566578


,0
Source Name,object
Gnai3,float64
Narf,float64
Cav2,float64
Klf6,float64
...,...
Zfp33b,float64
Klhl17,float64
sumcount,float64
dentotal,float64


In [3]:
# Renaming columns and adding env
unnormalized_data.rename(columns = {'Source Name':'sample', 'sumcount':'sumcount_thresh', 'dentotal':'dentotal_thresh', 'denEC':'denEC_thresh'}, inplace = True)
unnormalized_data.insert(len(unnormalized_data.columns) - 3, "env", "RR9")
print(unnormalized_data)

# Transform data
transformed_data = unnormalized_data.iloc[:, :-3]
transformed_data.head()

<ipython-input-3-6a7452d389d5>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  unnormalized_data.insert(len(unnormalized_data.columns) - 3, "env", "RR9")


   sample  Gnai3   Narf   Cav2   Klf6  Scmh1   Cox5a   Tbx2   Ngfr    Fer  \
0    GC15  329.0  768.0  365.0  333.0  786.0  1337.0  926.0  763.0  533.0   
1    GC16  269.0  660.0  310.0  325.0  599.0  1183.0  693.0  641.0  505.0   
2    GC17  175.0  546.0  213.0  287.0  530.0   912.0  602.0  607.0  390.0   
3    GC18  257.0  699.0  268.0  363.0  670.0  1047.0  692.0  602.0  473.0   
4    GC19  293.0  745.0  348.0  388.0  814.0  1139.0  875.0  786.0  567.0   
5     F15  256.0  586.0  243.0  297.0  534.0   918.0  662.0  593.0  442.0   
6     F16  253.0  631.0  268.0  326.0  592.0   932.0  602.0  462.0  383.0   
7     F17  194.0  555.0  281.0  265.0  503.0   798.0  573.0  583.0  384.0   
8     F18  285.0  722.0  366.0  424.0  785.0  1279.0  775.0  838.0  509.0   
9     F19  253.0  634.0  296.0  366.0  593.0  1020.0  624.0  713.0  462.0   
10    F20  289.0  797.0  296.0  322.0  753.0  1202.0  714.0  719.0  590.0   

    ...  Ahnak2  Rps6ka4  Arvcf   Muc4  Zfp33b  Klhl17  env  sumcount_thres

,sample,Gnai3,Narf,Cav2,Klf6,Scmh1,Cox5a,Tbx2,Ngfr,Fer,...,Afg2b,Tusc3,Lin54,Ahnak2,Rps6ka4,Arvcf,Muc4,Zfp33b,Klhl17,env
0,GC15,329.0,768.0,365.0,333.0,786.0,1337.0,926.0,763.0,533.0,...,133.0,1011.0,284.0,325.0,137.0,540.0,3.0,188.0,245.0,RR9
1,GC16,269.0,660.0,310.0,325.0,599.0,1183.0,693.0,641.0,505.0,...,91.0,838.0,251.0,426.0,82.0,333.0,766.0,171.0,202.0,RR9
2,GC17,175.0,546.0,213.0,287.0,530.0,912.0,602.0,607.0,390.0,...,63.0,542.0,227.0,327.0,76.0,297.0,18.0,128.0,136.0,RR9
3,GC18,257.0,699.0,268.0,363.0,670.0,1047.0,692.0,602.0,473.0,...,93.0,816.0,249.0,338.0,78.0,330.0,12.0,156.0,166.0,RR9
4,GC19,293.0,745.0,348.0,388.0,814.0,1139.0,875.0,786.0,567.0,...,121.0,950.0,309.0,425.0,85.0,476.0,66.0,173.0,200.0,RR9


In [4]:
deseq2data = transformed_data.iloc[:, 1:-1]
deseq2data.head()
data_mor = deseq2_norm(deseq2data)
data_mor = data_mor[0]
first_column = transformed_data.iloc[:, 0]
data_mor.insert(0, 'sample', first_column)
data_mor.iloc[:, 0] = data_mor.iloc[:, 0].apply(lambda x: f"{x}_mor")
data_mor['env'] = 'mor'
data_mor.head()

<ipython-input-4-90777af19882>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_mor.insert(0, 'sample', first_column)
<ipython-input-4-90777af19882>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_mor['env'] = 'mor'


,sample,Gnai3,Narf,Cav2,Klf6,Scmh1,Cox5a,Tbx2,Ngfr,Fer,...,Afg2b,Tusc3,Lin54,Ahnak2,Rps6ka4,Arvcf,Muc4,Zfp33b,Klhl17,env
0,GC15_mor,271.546129,633.882758,301.259384,274.847602,648.739385,1103.517249,764.290929,629.755917,439.921237,...,109.773967,834.447224,234.404561,268.244657,113.075440,445.698814,2.476105,155.169217,202.215203,mor
1,GC16_mor,262.077480,643.015378,302.022375,316.636360,583.585169,1152.556352,675.166147,624.504329,492.004191,...,88.658181,816.434677,244.540697,415.037199,79.889789,324.430486,746.287545,166.599439,196.801676,mor
2,GC17_mor,206.381247,643.909492,251.195461,338.465246,625.040349,1075.541129,709.951491,715.848098,459.935351,...,74.297249,639.192206,267.705961,385.638102,89.628427,350.258460,21.227785,150.953141,160.387712,mor
3,GC18_mor,256.286967,697.060662,267.256448,361.992876,668.141121,1044.095154,690.080083,600.329783,471.687687,...,92.741976,813.736051,248.309163,337.062237,77.783593,329.084433,11.966707,155.567186,165.539442,mor
4,GC19_mor,246.747039,627.394348,293.064742,326.750345,685.502012,959.197533,736.872556,661.922090,477.493416,...,101.898948,800.033061,260.221280,357.909527,71.581905,400.858670,55.581244,145.690231,168.428013,mor


In [5]:
data_log2 = log_scale_transform(transformed_data)
data_zscore = z_score_transform(transformed_data)
data_sqrt = sqrt_transform(transformed_data)
transformed_data  = pd.concat([data_log2, data_zscore , data_sqrt, data_mor], ignore_index=True)
print(transformed_data)

# Adding phenotypes to transformed data
last_three_columns = unnormalized_data.iloc[:, -3:]
transformed_data = pd.concat([transformed_data, last_three_columns], axis=1)
values_to_duplicate = transformed_data.iloc[0:11, -3:]
duplicated_values = pd.concat([values_to_duplicate] * 2, ignore_index=True).iloc[11:22, :]
transformed_data.iloc[11:22, -3:] = duplicated_values.values
transformed_data.iloc[22:33, -3:] = duplicated_values.values
transformed_data.iloc[33:44, -3:] = duplicated_values.values
print(transformed_data)

<ipython-input-1-1b717dfb4adb>:38: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  transformed_df.iloc[:, 1:-1] = transformed_df.iloc[:, 1:-1].applymap(lambda x: np.log2(x) if x > 0 else np.nan)
<ipython-input-1-1b717dfb4adb>:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  transformed_df.iloc[:, 1:-1] = transformed_df.iloc[:, 1:-1].applymap(lambda x: np.sqrt(x) if x >= 0 else np.nan)


         sample       Gnai3        Narf        Cav2        Klf6       Scmh1  \
0     GC15_log2    8.361944    9.584963    8.511753    8.379378    9.618386   
1     GC16_log2    8.071462    9.366322    8.276124    8.344296    9.226412   
2     GC17_log2    7.451211    9.092757    7.734710    8.164907    9.049849   
3     GC18_log2    8.005625    9.449149    8.066089    8.503826    9.388017   
4     GC19_log2    8.194757    9.541097    8.442943    8.599913    9.668885   
5      F15_log2    8.000000    9.194757    7.924813    8.214319    9.060696   
6      F16_log2    7.982994    9.301496    8.066089    8.348728    9.209453   
7      F17_log2    7.599913    9.116344    8.134426    8.049849    8.974415   
8      F18_log2    8.154818    9.495855    8.515700    8.727920    9.616549   
9      F19_log2    7.982994    9.308339    8.209453    8.515700    9.211888   
10     F20_log2    8.174926    9.638436    8.209453    8.330917    9.556506   
11  GC15_zscore   -0.135235   -0.044829   -0.127821 

In [6]:
#Turning binary threshold values into ints
transformed_data[transformed_data.columns[-3]] = transformed_data[transformed_data.columns[-3]].astype(int)
transformed_data[transformed_data.columns[-2]] = transformed_data[transformed_data.columns[-2]].astype(int)
transformed_data[transformed_data.columns[-1]] = transformed_data[transformed_data.columns[-1]].astype(int)
print(transformed_data)


         sample       Gnai3        Narf        Cav2        Klf6       Scmh1  \
0     GC15_log2    8.361944    9.584963    8.511753    8.379378    9.618386   
1     GC16_log2    8.071462    9.366322    8.276124    8.344296    9.226412   
2     GC17_log2    7.451211    9.092757    7.734710    8.164907    9.049849   
3     GC18_log2    8.005625    9.449149    8.066089    8.503826    9.388017   
4     GC19_log2    8.194757    9.541097    8.442943    8.599913    9.668885   
5      F15_log2    8.000000    9.194757    7.924813    8.214319    9.060696   
6      F16_log2    7.982994    9.301496    8.066089    8.348728    9.209453   
7      F17_log2    7.599913    9.116344    8.134426    8.049849    8.974415   
8      F18_log2    8.154818    9.495855    8.515700    8.727920    9.616549   
9      F19_log2    7.982994    9.308339    8.209453    8.515700    9.211888   
10     F20_log2    8.174926    9.638436    8.209453    8.330917    9.556506   
11  GC15_zscore   -0.135235   -0.044829   -0.127821 

In [7]:
# Rename the last column to "thresh" when creating df1
df1 = transformed_data.iloc[:, :-2]
df1.iloc[:, 0] = df1.iloc[:, 0].astype(str) + "_sumcount"
df1.iloc[:, -2] = df1.iloc[:, -2].astype(str) + "_sumcount"
df1.rename(columns={df1.columns[-1]: 'thresh'}, inplace=True)

# Create df2 and add "dentotal" to the values in the first column
df2 = transformed_data.iloc[:, :-1].drop(transformed_data.columns[-3], axis=1)
df2.iloc[:, 0] = df2.iloc[:, 0].astype(str) + "_dentotal"
df2.iloc[:, -2] = df2.iloc[:, -2].astype(str) + "_dentotal"
df2.rename(columns={df2.columns[-1]: 'thresh'}, inplace=True)

# Create df3 and add "denEC" to the values in the first column
columns_to_drop = transformed_data.columns[[-3, -2]]
df3 = transformed_data.drop(columns=columns_to_drop)
df3.iloc[:, 0] = df3.iloc[:, 0].astype(str) + "_denEC"
df3.iloc[:, -2] = df3.iloc[:, -2].astype(str) + "_denEC"
df3.rename(columns={df3.columns[-1]: 'thresh'}, inplace=True)

# Concatenate the results
result = pd.concat([df1, df2, df3], ignore_index=True)
# Display the result
print(result)

                 sample       Gnai3        Narf        Cav2        Klf6  \
0    GC15_log2_sumcount    8.361944    9.584963    8.511753    8.379378   
1    GC16_log2_sumcount    8.071462    9.366322    8.276124    8.344296   
2    GC17_log2_sumcount    7.451211    9.092757    7.734710    8.164907   
3    GC18_log2_sumcount    8.005625    9.449149    8.066089    8.503826   
4    GC19_log2_sumcount    8.194757    9.541097    8.442943    8.599913   
..                  ...         ...         ...         ...         ...   
127       F16_mor_denEC  279.157518  696.238711  295.708359  359.704944   
128       F17_mor_denEC  250.275938  715.995597  362.513086  341.871771   
129       F18_mor_denEC  247.823670  627.819964  318.257765  368.692056   
130       F19_mor_denEC  252.935352  633.837996  295.924364  365.906477   
131       F20_mor_denEC  258.462781  712.784900  264.723125  287.975832   

          Scmh1        Cox5a        Tbx2        Ngfr         Fer  ...  \
0      9.618386    10.3847

In [8]:
#result.to_pickle('/content/drive/MyDrive/Colab Notebooks/Capstone/RR9_HNE_combined_thresh_log_zscore_sqrt_mor.pkl')

In [12]:
def column_names_to_txt(df, file_name):
    """
    Writes only the column names of a dataframe to a text file.

    Parameters:
    df (pd.DataFrame): The dataframe whose column names will be written.
    file_name (str): The name of the text file to create.
    """
    with open(file_name, 'w') as file:
        file.write("\n".join(df.columns.tolist()))  # Write all column names, each on a new line

# Save the dataframe columns to a text file
# Manually remove sample, env, and thresh columns from text file
column_names_to_txt(result, "/content/drive/MyDrive/Colab Notebooks/Capstone/background_gene_list.txt")